# Fraud Detection using Variational AutoEncoder

1. Raw Dataset
2. Visualize Preprocessed Transaction Features
3. Train a Variational Autoencoder
4. Visualize Latent Representations
5. VAE vs SVM
6. Evaluation on Test Set
7. Limitations

Source: https://www.kaggle.com/code/hone5com/fraud-detection-with-variational-autoencoder